In [15]:
RK = os.environ.get('RAPID_KEY')

!{sys.executable} -m pip install requests pandas numpy scipy plotly chart_studio
import requests

Defaulting to user installation because normal site-packages is not writeable


In [65]:
repo = "https://github.com/expressjs/express"
payload = {"urls": [repo]}
headers = {'X-RapidApi-Key': RK, 'X-RapidApi-Host': 'lowendinsight2.p.rapidapi.com', 'Content-Type': 'application/json'}
r = requests.post('https://lowendinsight2.p.rapidapi.com/analyze/', data=json.dumps(payload), headers=headers)

import json
import time

j = json.loads(r.content)
print(j)
while j['state'] == "incomplete":
  r = requests.get("https://lowendinsight2.p.rapidapi.com/analyze/{}".format(j["uuid"]))
  print(r)
  j = json.loads(r.content)
  time.sleep(2)
  print("not complete yet, sleeping...")
    

{'uuid': 'e9d906e4-968f-11ed-8616-06aa01f57570', 'state': 'complete', 'report': {'repos': [{'header': {'uuid': 'e7e4d598-968f-11ed-a2ed-5641c3ad0f84', 'start_time': '2023-01-17T17:53:49.490472Z', 'source_client': 'lei_worker', 'repo': 'https://github.com/expressjs/express', 'library_version': '0.8.1', 'end_time': '2023-01-17T17:53:53.335586Z', 'duration': 4}, 'data': {'risk': 'medium', 'results': {'top10_contributors': [{'name': 'Tj Holowaychuk', 'merges': 343, 'last_contribution_date': '2012-02-16T08:17:59-08:00', 'email': 'tj@vision-media.ca', 'contributions': 3881}, {'name': 'Douglas Christopher Wilson', 'merges': 54, 'last_contribution_date': '2022-10-08T16:11:42-04:00', 'email': 'doug@somethingdoug.com', 'contributions': 1147}, {'name': 'Jonathan Ong', 'merges': 19, 'last_contribution_date': '2014-06-05T19:32:32-04:00', 'email': 'jonathanrichardong@gmail.com', 'contributions': 84}, {'name': 'Roman Shtylman', 'merges': 16, 'last_contribution_date': '2014-05-12T15:45:54-04:00', 'ema

In [66]:


results = j["report"]["repos"][0]["data"]["results"]
#print(results)
total_commits = j["report"]["repos"][0]["data"]["git"]["total_commits_on_default_branch"]
top10 = results["top10_contributors"]
#print(top10)

for c in top10:
    #print("{} [{}]: {} {}".format(c["name"], c["email"], c["contributions"], round((c["contributions"]/total_commits)*100,2)))
    c.update({'contribution_percentage': round((c["contributions"]/total_commits)*100,2)})

import pandas as pd
import numpy as np
import scipy as sp
import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.figure_factory as ff

df = pd.DataFrame.from_dict(top10)
df = df.drop('email', axis=1)
table = ff.create_table(df)

#plotly.offline.plot(table, filename='lei-table1.png')
table.update_layout(width=1024)
table.show()

In [67]:
import plotly.express as px
fig = px.bar(x=df['name'].to_list(), y=df['contributions'].to_numpy(), title=repo, labels=dict(x="Contributor", y="Contributions"))
fig.update_layout(width=1024)
fig.show()